In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

In [2]:
training = 'https://raw.githubusercontent.com/KCL-Health-NLP/nlp_examples/master/pubmed/dev.txt'
train_data = pd.read_csv(training, sep='\t', header=None)

In [3]:
train_data.columns = ['number', 'label', 'text']
train_data.head()

,number,label,text
0,24290286,BACKGROUND,IgE sensitization to Aspergillus fumigatus and...
1,24290286,BACKGROUND,It is not clear whether these patients would b...
2,24290286,OBJECTIVE,We sought to determine whether a 3-month cours...
3,24290286,METHODS,Asthmatic patients who were IgE sensitized to ...
4,24290286,METHODS,Primary outcomes were improvement in quality o...


In [4]:
train_data.shape

(30212, 3)

In [5]:
train_data['label'].value_counts()

,count
label,
METHODS,9964
RESULTS,9841
CONCLUSIONS,4582
BACKGROUND,3449
OBJECTIVE,2376


There more than 30,000 rows in the dataset, however due to time constarin we are going to use only 1000 data points.

In [6]:
grouped = train_data.groupby('label')

In [7]:
df_background = grouped.get_group('BACKGROUND')
df_methods = grouped.get_group('METHODS')
df_results = grouped.get_group('RESULTS')
df_conclusions = grouped.get_group('CONCLUSIONS')
df_objective = grouped.get_group('OBJECTIVE')

In [8]:
df_background_200 = df_background.iloc[:200]
df_methods_200 = df_methods.iloc[:200]
df_results_200 = df_results.iloc[:200]
df_conclusions_200 = df_conclusions.iloc[:200]
df_objective_200 = df_objective.iloc[:200]






In [9]:
result_df = pd.concat([df_background_200, df_methods_200, df_results_200, df_conclusions_200, df_objective_200])  # Concatenate and reset index


In [11]:
result_df.shape

(1000, 3)

In [12]:
result_df.head()

,number,label,text
0,24290286,BACKGROUND,IgE sensitization to Aspergillus fumigatus and...
1,24290286,BACKGROUND,It is not clear whether these patients would b...
10,24464531,BACKGROUND,"Opioid antagonists ( e.g. , naltrexone ) and p..."
11,24464531,BACKGROUND,The use of higher doses to achieve greater eff...
12,24464531,BACKGROUND,Combining naltrexone and alprazolam might safe...


In [13]:
result_df['label'].value_counts()

,count
label,
BACKGROUND,200
METHODS,200
RESULTS,200
CONCLUSIONS,200
OBJECTIVE,200


Loading pre-trained BERT model and tokenizer

In [14]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [15]:
def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors='tf', padding=True, truncation=True, max_length=128)
    outputs = bert_model(inputs)
    cls_embedding = outputs.last_hidden_state[:, 0, :].numpy()  # Use [CLS] token embedding
    return cls_embedding

In [16]:
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(result_df.label)

In [17]:
texts = result_df.text.to_numpy()

In [18]:
len(texts)

1000

Generate BERT embeddings for each text

In [20]:
embeddings = np.array([get_bert_embedding(text) for text in texts])


Train test split

In [21]:
X_train, X_test, y_train, y_test = train_test_split(embeddings, encoded_labels, test_size=0.2, random_state=42)


In [23]:
svm_clf = SVC(kernel='linear')  # 'ovr' stands for one-vs-rest
svm_clf.fit(np.squeeze(X_train), y_train)

SVC(kernel='linear')

In [24]:
y_pred = svm_clf.predict(np.squeeze(X_test))

In [25]:
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))


              precision    recall  f1-score   support

  BACKGROUND       0.48      0.70      0.57        33
 CONCLUSIONS       0.66      0.66      0.66        38
     METHODS       0.89      0.83      0.86        48
   OBJECTIVE       0.76      0.57      0.65        44
     RESULTS       0.86      0.84      0.85        37

    accuracy                           0.72       200
   macro avg       0.73      0.72      0.72       200
weighted avg       0.74      0.72      0.73       200

